In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from datetime import datetime
import torch.nn.functional as F


In [2]:
#import data
train_data=datasets.CIFAR10(root='/content/sample_data',train=True, transform=transforms.ToTensor(),download=True)
test_data=datasets.CIFAR10(root='/content/sample_data',train=False, transform=transforms.ToTensor(),download=True)
train_loader=DataLoader(dataset=train_data,batch_size=128,shuffle=True)
test_loader=DataLoader(dataset=test_data,batch_size=128,shuffle=False)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting /content/sample_data/cifar-10-python.tar.gz to /content/sample_data
Files already downloaded and verified


In [3]:
# #Classes
k=len(set(train_data.targets))
# n_input_channels=train_data.data.shape[3]


In [4]:
#Create model
class CNN(nn.Module):
  def __init__(self,k):
    super(CNN,self).__init__()
    # define the conv layers
    self.conv1 = nn.Sequential(
        nn.Conv2d(3, 32, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(32),
        nn.Conv2d(32, 32, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(32),
        nn.MaxPool2d(2),
    )
    self.conv2 = nn.Sequential(
        nn.Conv2d(32, 64, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(64),
        nn.Conv2d(64, 64, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(64),
        nn.MaxPool2d(2),
    )
    self.conv3 = nn.Sequential(
        nn.Conv2d(64, 128, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(128),
        nn.Conv2d(128, 128, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(128),
        nn.MaxPool2d(2),
    )

    # Useful: https://pytorch.org/docs/stable/nn.html#torch.nn.MaxPool2d
    # H_out = H_in + 2p - 2 --> p = 1 if H_out = H_in
    
    # Easy to calculate output
    # 32 > 16 > 8 > 4

    # define the linear layers
    self.fc1 = nn.Linear(128 * 4 * 4, 1024)
    self.fc2 = nn.Linear(1024, k)
  
  def forward(self, x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.conv3(x)
    x = x.view(x.size(0), -1)
    x = F.dropout(x, p=0.5)
    x = F.relu(self.fc1(x))
    x = F.dropout(x, p=0.2)
    x = self.fc2(x)
    return x


model=CNN(k)

criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters())

In [ ]:
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
epoch=15

for iter in range(epoch):
  if (iter-1)%10==0 or iter==0:
    t0=datetime.now()
  for inputs,targets in train_loader:
      
      #move data to GPU
      inputs,targets=inputs.to(device),targets.to(device)
      
      #zero the grads
      optimizer.zero_grad()

      #forward propagation
      y_hats=model(inputs)
      loss=criterion(y_hats,targets)

      #backward propagation
      loss.backward()
      optimizer.step()

  # for inputs,targets in test_loader:
  #   #move data to GPU
  #   inputs,targets=inputs.to(device),targets.to(device)
    
  #   y_hats=model(inputs)
  #   test_loss=criterion(y_hats,targets)

  
  if epoch%10!=0:
        print("Epoch " + str(iter)+"/" + str(epoch)," |  Train_Loss = ",loss.data,'Duration =',datetime.now()-t0)

In [ ]:
#calculate The Accuracy for all batches
n_corrects=0
n_samples=0
for inputs,targets in train_loader:
  inputs,targets=inputs.to(device),targets.to(device)
  probs=model(inputs)
  _,preds=torch.max(probs,1)
  n_corrects+=torch.sum(preds==targets)
  n_samples+=targets.shape[0]

train_accuracy=n_corrects/n_samples

n_corrects=0
n_samples=0
for inputs,targets in test_loader:
  inputs,targets=inputs.to(device),targets.to(device)
  probs=model(inputs)
  _,preds=torch.max(probs,1)
  n_corrects+=torch.sum(preds==targets)
  n_samples+=targets.shape[0]

test_accuracy=n_corrects/n_samples

print('Train Accuray = ',train_accuracy)
print('Test Accuracy = ',test_accuracy)

Train Accuray =  tensor(0.7856, device='cuda:0')
Test Accuracy =  tensor(0.6606, device='cuda:0')


In [ ]:
from torchsummary import summary
summary(model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             896
              ReLU-2           [-1, 32, 32, 32]               0
       BatchNorm2d-3           [-1, 32, 32, 32]              64
            Conv2d-4           [-1, 32, 32, 32]           9,248
              ReLU-5           [-1, 32, 32, 32]               0
       BatchNorm2d-6           [-1, 32, 32, 32]              64
         MaxPool2d-7           [-1, 32, 16, 16]               0
            Conv2d-8           [-1, 64, 16, 16]          18,496
              ReLU-9           [-1, 64, 16, 16]               0
      BatchNorm2d-10           [-1, 64, 16, 16]             128
           Conv2d-11           [-1, 64, 16, 16]          36,928
             ReLU-12           [-1, 64, 16, 16]               0
      BatchNorm2d-13           [-1, 64, 16, 16]             128
        MaxPool2d-14             [-1, 6